![BAE logo](images/bae_logo.png)

# Hands-On Exercise 12.3: The Template Method Pattern

## Objective

In this exercise, you will use the Template Method pattern to reduce code duplication in an algorithm; specifically, the `KnuthShuffler` and the `ModifiedKnuthShuffler` are almost the same.

#### Review the two similar shufflers.

Open the solution `C:\Course\510D\Exercises\Ex123\Blackjack.sln`.

This exercise starts with the completed bonus from the previous exercise.

Run all tests to ensure everything is working before modifications are made.

Since the changes to be made are a refactoring, no new tests will be required.

In the `CardLib` project, open `KnuthShuffler.cs` for editing.

Insert the layout for a new public abstract class called `KnuthShuffleLogic` that implements the interface `IShuffler`.

Cut the entire `Shuffle(...)` method from `KnuthShuffler` and paste it into `KnuthShuffleLogic`.

|<font color="red">**Make sure you cut, not copy the method.**</font>

The `Shuffle(...)` method in `KnuthShuffleLogic` will be the base invariant algorithm.

Modify the `KnuthShuffler` class to inherit from `KnuthShuffleLogic` rather than implement the interface `IShuffler`.

You can run all tests for green lights, since at this point the `KnuthShuffleLogic` is the same as it was in the `KnuthShuffler`.

Move to the `ModifiedKnuthShuffler` class and delete the `Shuffle()` method entirely.

Indicate that it inherits `KnuthShuffleLogic` rather than implements `IShuffler`.

Your C# code might look something like this:

```C#
public abstract class KnuthShuffleLogic : IShuffler
{
    public void Shuffle(List<PlayingCard> deck)
    {
        for (int i = 0; i < deck.Count; i++)
        {
            int swapIndex = RandomNumber.Next(0, deck.Count);
            PlayingCard temp = deck[swapIndex]; 
            deck[swapIndex] = deck[i]; 
            deck[i] = temp;
        }
    }
}

public class KnuthShuffler : KnuthShuffleLogic
{
}

public class ModifiedKnuthShuffler : KnuthShuffleLogic
{
}
```

Run all tests. They will give green lights; but that will be misleading since both the `KnuthShuffler` and the `ModifiedKnuthShuffler` are using the same algorithm.

#### Apply the Template Methods pattern to restore different algorithms for the two shufflers.

Into the `KnuthShufflerLogic` class, add a protected abstract method with the specification `int Swix(int i)`. Swix is short for Swap Index.

Your C# code might look something like this:

```C#
protected abstract int Swix(int i);
```

Change the statement `RandomNumber.Next(0, deck.Count);` to be `RandomNumber.Next(Swix(i), deck.Count);`.

Note this is using the Hollywood principle. "Don't call us. We'll call you.".

Override `Swix(...)` in the `KnuthShuffler` class to return the fixed value `0`.

Override `Swix(i)` in the `ModifiedKnuthShuffler`, but return `i` not `0`.

Your C# code might look something like this:

```C#
public class KnuthShuffler : KnuthShuffleLogic
{
    protected override int Swix(int i)
    {
        return 0;
    }
}

public class ModifiedKnuthShuffler : KnuthShuffleLogic
{
    protected override int Swix(int i)
    {
        return i;
    }
}
```

Run all tests to get green lights.

Our refactoring has reduced code duplication but did not cause the externally visible result to change.

Now would be a good time to run all tests and the application.

## Congratulations! You have used a Template Method to reduce code duplication. Carry on to the bonus if you have more time.

# Bonus (Optional)

Add support for the shuffler strategy to accept a shuffle method delegate in addition to an `IShuffler` object.

By convention, Microsoft allows both an interface and a delegate when it implements a strategy. For example, the `Sort()` method of a list accepts both an `IComparer<T>` and a `Comparer` delegate.

#### Modify the card deck internally to use a delegate.

Open `CardDeck.cs` for editing and before the class declaration, but inside the namespace, add a public delegate specification called `ShuffleMethod`. It should accept a `List<PlayingCard>` as a parameter and return `void`.
    
Note we are using a full delegate specification rather than a generic `Func<T,R>`. This is because `Func<T,R>` does not support a return type of `void`.

You delegate should look like...

```C#
public delegate void ShufflerMethod(List<PlayingCard> deck);
```

Comment out the line of code that declares the field `currentShuffler`. Replace it with a field called `CurrentShufflerMethod` of type `ShufflerMethod`. Internally, the `CardDeck` class will now use the delegate.

Your code should look something like...

```C#
// private IShuffler currentShuffler = new KnuthShuffler();
private ShufflerMethod CurrentShufflerMethod;
```

Scroll to the default `Shuffle()` method. Fix the error by making a call to `CurrentShufflerMethod(deck);`.

Move to the `ChangeShuffler(...)` method and modify it to assign `CurrentShufflerMethod = shuffler.Shuffle;`.

Run all tests.

There will be lots of red lights because the CurrentShuffleMethod is not properly initialized.

Modify the `CardDeck()` constructor to initialize the `CurrentShufflerMethod` delegate. To do this:

- Construct an instance of a `KnuthShuffler`
- Assign its `Shuffle` method to the `CurrentShufflerMethod` field

Your code should look like...

```C#
IShuffler shuffler = new KnuthShuffler(); CurrentShufflerMethod = shuffler.Shuffle;
```

Run all tests. You should get all green lights.

#### Add support for directly passing in a delegated shuffler method when changing shufflers.

Find the method `ChangeShufflers(IShuffler shuffler)` just before it adds an overload `ChangeShuffler(ShufflerMethod method)`.

Inside this method assign method to CurrentShufflerMethod

Your new method should look like...

```C#
public void ChangeShuffler(ShufflerMethod method)
{
    CurrentShufflerMethod = method;
}
```

In a similar fashion, add an overload for `Shuffle(ShufflerMethod method)`. Do not assign the method to the `CurrentShufflerMethod`, rather, just call `method(deck)` to do the shuffling.

Your overload should look like...

```C#
public CardDeck Shuffle(ShufflerMethod method)
{
    method(deck); 
    return this;
}
```

Run all tests and get green lights.

Note though there are no tests for changing shufflers by delegate. Add some tests now.

Open `CardDeckTests.cs` for editing and find the `ShuffleStrategyTest()`. Highlight and copy this method.

Paste it back into the `CardDeckTests` class and rename it `ShuffleDelegateStrategyTests`.

Inside this method, create an instance of a `KnuthShuffler` and assign it to an `IShuffler` variable called `shuffler`.

In the call to `deck.Shuffle(new KnuthShuffler())` replace it with `deck.Shuffle(shuffler.Shuffle)`.

Your code should look like...

```C#
IShuffler shuffler = new KnuthShuffler();
deck.Shuffle(shuffler.Shuffle);
```

Run all tests to get green lights.

Play the game a bit to ensure all appears correct. Try changing shufflers too.

Your delegate-based strategy is now working and tested.

## Congratulations! You have completed the bonus.